## Data cleansing and JSON creation

La neteja de les dades està dividida en dos fitxers. El primer fitxer és aquest; el segon és RendaEquipamentsServeissocials.ipynb.

Datasets utilitzats en aquest fitxer:
1.  [Equipaments educacio csv](https://opendata-ajuntament.barcelona.cat/data/ca/dataset/educacio-ensenyament-reglat/resource/4809b3ab-a500-4086-bc7d-52bdcd9f2e0e)
2.  [Educacio ensenyament infantil](https://opendata-ajuntament.barcelona.cat/data/ca/dataset/educacio-ensenyament-infantil/resource/114a72b3-2c9c-40c6-a7e6-d85dca704726)
3. [Educacio ensenyament regla](https://opendata-ajuntament.barcelona.cat/data/ca/dataset/educacio-ensenyament-reglat/resource/4809b3ab-a500-4086-bc7d-52bdcd9f2e0e)
4. [Estudiants barri](https://opendata-ajuntament.barcelona.cat/data/ca/dataset/pad_mdbas_niv-educa-esta_sexe)

Datasets utilizats en RendaEquipamentsServeissocials:

1. [Equipaments associacions](https://opendata-ajuntament.barcelona.cat/data/ca/dataset/equipament-associacions/resource/796ee1c2-975a-4d68-8753-52df8c2fc304)
2. [Equipaments de serveis socials ](https://opendata-ajuntament.barcelona.cat/data/ca/dataset/equipament-serveis-socials)
3. [Renda](https://opendata-ajuntament.barcelona.cat/data/ca/dataset/atles-renda-bruta-per-persona)

Els dos fitxers tracten les dades i creen els JSON que després utilitzem com a context per al ChatGPT amb l'ajuda de l'API d'OpenAI.

El fitxer Data_Cleaning conté una funció per carregar les dades amb el format pertinent, llegir-les amb l'encoding necessari i fer-ne la neteja quan convingui. La major part del codi ha estat continguda en una classe, ja que al principi teníem una gran quantitat de cel·les amb les quals treballàvem de forma individual cada conjunt de dades i, per tal de reutilitzar i fer el codi més fàcil d'entendre, hem decidit crear una classe per tractar amb els conjunts de dades.

In [1]:
# Paths data
import pandas as pd
import os

class CleanData:
    def __init__(self, path, process, encoding='utf-8'):
        self.path = path
        self.raw_data = pd.read_csv(self.path, sep=',', encoding=encoding)
        if process:
            self.process_dataset()
        else:
            self.dataset = self.raw_data

    def process_dataset(self):
        dataset = self.raw_data.loc[:, ['name', 'addresses_district_id', 'addresses_district_name', 'addresses_neighborhood_id', 'addresses_neighborhood_name']]
        dataset = dataset.dropna(axis=0, inplace=False)
        self.dataset = dataset
    
    def rename_column(self, col, new_col):
        self.dataset = self.dataset.rename(columns={col: new_col})

# PATH ALS FITXERS EN LOCAL
Escoles_infantils_path = r'C:\Users\34644\Desktop\JUMP2DIGITAL\Datasets\opendatabcn_educacio_ensenyament_infantil.csv'
Escoles_primaries_cursos_reglats_path = r'C:\Users\34644\Desktop\JUMP2DIGITAL\Datasets\opendatabcn_educacio_ensenyament_reglat.csv'
Equipaments_path = r'C:\Users\34644\Desktop\JUMP2DIGITAL\Datasets\opendatabcn_llista-equipaments_educacio-csv.csv'
Estudiante_barrios_path = r'C:\Users\34644\Desktop\JUMP2DIGITAL\Datasets\estudiantes_barrios.csv'
Llistat_equipaments_path = r'C:\Users\34644\Desktop\JUMP2DIGITAL\Datasets\opendatabcn_llista-equipaments_serveis-socials-csv.csv'

c:\Users\34644\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\34644\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\34644\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [115]:
# Funcio que crear un json en vase a un dataset i les columnes pertinents.
def create_json(json_path, dataset, type_function, grup_by = 'Codi_Bari', column_name = 'Valor'):
    import json
    # Assuming Escoles_infantils is your DataFrame
    grouped = dataset.groupby(grup_by)

    # Using dictionary comprehension
    new_data = {
        neighborhood: type_function(group[column_name].unique())
        for neighborhood, group in grouped
    }

    with open(json_path, 'w') as fp:
        json.dump(new_data, fp)


In [116]:
Escoles_infantils                   = CleanData(Escoles_infantils_path, process = True, encoding='UTF-16')
Escoles_primaries_cursos_reglats    = CleanData(Escoles_primaries_cursos_reglats_path,process = True, encoding='UTF-16')
Equipaments                         = CleanData(Equipaments_path, process = True, encoding='UTF-16')
Estudiante_barrios                  = CleanData(Estudiante_barrios_path, process=False)

Escoles_infantils.rename_column('addresses_neighborhood_id', 'Codi_barri')
Escoles_primaries_cursos_reglats.rename_column('addresses_neighborhood_id', 'Codi_barri')
Equipaments.rename_column('addresses_neighborhood_id', 'Codi_barri')

In [117]:
create_json('escoles_infantils.json', Escoles_infantils.dataset, list, 'Codi_barri', 'name')
create_json('estudis_reglamentaris.json', Escoles_primaries_cursos_reglats.dataset, list, 'Codi_barri', 'name')
create_json('equipaments.json', Equipaments.dataset, list, 'Codi_barri', 'name')
create_json('numero_estudiants.json', Estudiante_barrios.dataset, int, 'Codi_Barri', 'Valor')

C:\Users\34644\AppData\Local\Temp\ipykernel_16348\2023641169.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  neighborhood: type_function(group[column_name].unique())
